In [ ]:
import tensorflow as tf
import numpy as np
import sys
import h5py
import random
import math
from math import sqrt

#thanks to https://gist.githubusercontent.com/kukuruza/03731dc494603ceab0c5/raw/3d708320145df0a962cfadb95b3f716b623994e0/gist_cifar10_train.py
def put_kernels_on_grid (kernel, grid_Y, grid_X, pad = 1):

    '''Visualize conv. features as an image (mostly for the 1st layer).
    Place kernel into a grid, with some paddings between adjacent filters.

    Args:
      kernel:            tensor of shape [Y, X, NumChannels, NumKernels]
      (grid_Y, grid_X):  shape of the grid. Require: NumKernels == grid_Y * grid_X
                           User is responsible of how to break into two multiples.
      pad:               number of black pixels around each filter (between them)

    Return:
      Tensor of shape [(Y+2*pad)*grid_Y, (X+2*pad)*grid_X, NumChannels, 1].
    '''

    x_min = tf.reduce_min(kernel)
    x_max = tf.reduce_max(kernel)

    kernel1 = (kernel - x_min) / (x_max - x_min)

    # pad X and Y
    x1 = tf.pad(kernel1, tf.constant( [[pad,pad],[pad, pad],[0,0],[0,0]] ), mode = 'CONSTANT')

    # X and Y dimensions, w.r.t. padding
    Y = kernel1.get_shape()[0] + 2 * pad
    X = kernel1.get_shape()[1] + 2 * pad

    channels = kernel1.get_shape()[2]

    # put NumKernels to the 1st dimension
    x2 = tf.transpose(x1, (3, 0, 1, 2))
    # organize grid on Y axis
    x3 = tf.reshape(x2, tf.stack([grid_X, Y * grid_Y, X, channels])) #3

    # switch X and Y axes
    x4 = tf.transpose(x3, (0, 2, 1, 3))
    # organize grid on X axis
    x5 = tf.reshape(x4, tf.stack([1, X * grid_X, Y * grid_Y, channels])) #3

    # back to normal order (not combining with the next step for clarity)
    x6 = tf.transpose(x5, (2, 1, 3, 0))

    # to tf.image_summary order [batch_size, height, width, channels],
    #   where in this case batch_size == 1
    x7 = tf.transpose(x6, (3, 0, 1, 2))

    # scale to [0, 255] and convert to uint8
    return tf.image.convert_image_dtype(x7, dtype = tf.uint8)

def main():
    #sess = tf.InteractiveSession()
    
    trainfile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_Coch.hdf5"
    train_averagefile = "/om/user/ribeirop/audiosetDL/balanced_stripped/balanced_train_segments_average.npy"
    testfile =  trainfile#"/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"
    test_averagefile = train_averagefile #"/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_average.npy"
    trainset = h5py.File(trainfile)
    testset = h5py.File(testfile)
    
    #TODO: Subtract this when doing the model
    train_ave_coch = np.load(train_averagefile)
    test_ave_coch = np.load(test_averagefile)
    
    COCHLEAGRAM_LENGTH = int(342000/1) # full is 342000
    conv1_strides = 3
    conv2_strides = 2
    conv3_strides = 1
    conv4_strides = 1
    conv5_strides = 1
    
    trainsize = trainset.attrs.get("size")
    testsize = testset.attrs.get("size")
    
    b = 0
    batchsize = 1
    
    final_filter = 512
    full_length = final_filter * math.ceil(COCHLEAGRAM_LENGTH/171/conv1_strides/conv2_strides/conv3_strides/conv4_strides/conv5_strides/4)* math.ceil(171/conv1_strides/conv2_strides/conv3_strides/conv4_strides/conv5_strides/4)
    
    nets = {}

    with tf.Graph().as_default():
        with tf.name_scope('input'):
            nets['input_to_net'] = tf.placeholder(tf.float32, [None,COCHLEAGRAM_LENGTH], name='input_to_net')
            nets['actual_labels'] = tf.placeholder(tf.float32, [None, 527], name='actual_labels')
            nets['label_indeces'] = tf.placeholder(tf.int32, [None, 15], name='label_indeces')
            
            
            nets['accuracy'] = tf.placeholder(tf.float32, (), name='acc')
            nets['accsum'] = tf.summary.scalar("accuracy",nets['accuracy'])
            
            
            nets['reshapedCoch'] = tf.reshape(nets['input_to_net'],[-1,171,int(COCHLEAGRAM_LENGTH/171),1], name='reshape_input')

        #sess.run(tf.global_variables_initializer())
        #thing = sess.run(reshapedCoch,feed_dict={input_to_net:trainset["/coch"][0:2],actual_labels:trainset["/labels"][0:2]})



        with tf.variable_scope('image_filters') as scope:
            tf.get_variable_scope().reuse_variables()
            conv1_Weights = weight_variable([9,9,1,96])
            
            grid = put_kernels_on_grid(conv1_Weights,16,6)
            nets['conv1_weight_image'] = tf.summary.image('conv1/kernels', grid, max_outputs=3)
            
            nets['conv1'] = conv2d(nets['reshapedCoch'], Weights = conv1_Weights, bias = bias_variable([96]), strides=conv1_strides, name='conv1')
            nets['maxpool1'] = maxpool2x2(nets['conv1'],k=2 , name='maxpool1')


            
            #nets['conv1_image'] = tf.reshape(nets['conv1'],[])

        with tf.name_scope('conv2'):
            nets['conv2'] = conv2d(nets['maxpool1'], Weights = weight_variable([5,5,96,256]), bias = bias_variable([256]), strides=conv2_strides, name='conv2')
            nets['maxpool2'] = maxpool2x2(nets['conv2'],k=2 , name='maxpool2')

        with tf.name_scope('conv3'):
            nets['conv3'] = conv2d(nets['maxpool2'], Weights = weight_variable([3,3,256,512]), bias = bias_variable([512]), strides=conv3_strides, name='conv3')

        with tf.name_scope('conv4'):
            nets['conv4'] = conv2d(nets['conv3'], Weights = weight_variable([3,3,512,1024]), bias = bias_variable([1024]), strides=conv4_strides, name='conv4')        

        with tf.name_scope('conv5'):
            nets['conv5'] = conv2d(nets['conv4'], Weights = weight_variable([3,3,1024,final_filter]), bias = bias_variable([final_filter]),strides=conv5_strides, name='conv5')             
            nets['flattened'] = tf.reshape(nets['conv5'], [-1, full_length])

        with tf.name_scope('fc_1'):
            W_fc1 = weight_variable([full_length, 1024]) # 4,959,232
            b_fc1 = bias_variable([1024])
            nets['fc_1'] = tf.nn.relu(tf.matmul(nets['flattened'], W_fc1) + b_fc1)

            keep_prob = tf.placeholder(tf.float32)
            nets['h_fc1_drop'] = tf.nn.dropout(nets['fc_1'], keep_prob)

        with tf.name_scope('fc_2'):
            W_fc2 = weight_variable([1024, 527])
            b_fc2 = bias_variable([527])
        
        with tf.name_scope('predictions'):
            nets['predicted_labels'] = tf.matmul(nets['h_fc1_drop'], W_fc2) + b_fc2

        with tf.variable_scope("eval"):
            nets['cross_entroy'] = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels =nets['actual_labels'],logits = nets['predicted_labels'] ))
            nets['train_step'] = tf.train.AdamOptimizer(1e-4).minimize(nets['cross_entroy'])

            #num = np.sum(nets['actual_labelsi'])
            #nets['accuracy'] = tf.metrics.mean(tf.nn.in_top_k(nets['predicted_labels'],nets['actual_labelsi'],2))

            #calculating accuracys

            _,nets['indeces'] = tf.nn.top_k(nets['predicted_labels'],15)
            nets['numCorrect'] = tf.shape(tf.sets.set_intersection(nets['indeces'],nets['label_indeces'],False).values)[0]
        
        merged = tf.summary.merge_all()
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            train_writer = tf.summary.FileWriter('./TB/train5', sess.graph)
            for i in range(sys.maxsize):
                if(i%10 == 0 ):
                    print(i)                
                #get random batch
                start = random.randint(0,trainsize-batchsize)
                end = start+batchsize     
                thing = sess.run(nets['train_step'],feed_dict={nets['input_to_net']:trainset["/coch"][start:end][:,0:COCHLEAGRAM_LENGTH]-train_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:trainset["/labels"][start:end],keep_prob:.5})
                
                
                if(i%200 == 0):
                    
                    total_count = 0
                    correct = 0
                    
                    for x in range(100):
                        #get random batch
                        start = random.randint(0,testsize-batchsize)
                        end = start+batchsize
                        trainBatchLabels = testset["/labels"][start:end]
    
                        indeces_list = []

                        for row in trainBatchLabels:
                            indeces= np.where(row==1)[0]
                            total_count = total_count + len(indeces)
                            indeces_list.append(np.pad(indeces,[0,15-len(indeces)],mode = 'constant',constant_values=-1))
                        
    
                        correct = correct + sess.run(nets['numCorrect'],feed_dict={nets['input_to_net']:testset["/coch"][start:end][:,0:COCHLEAGRAM_LENGTH]-test_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:testset["/labels"][start:end],nets['label_indeces']:indeces_list,keep_prob:1})
                        potato = sess.run(nets['indeces'],feed_dict={nets['input_to_net']:testset["/coch"][start:end][:,0:COCHLEAGRAM_LENGTH]-test_ave_coch[0:COCHLEAGRAM_LENGTH],nets['actual_labels']:testset["/labels"][start:end],nets['label_indeces']:indeces_list,keep_prob:1})

                    summary = sess.run(nets['accsum'],feed_dict={nets['accuracy']:correct/total_count})
                    train_writer.add_summary(summary, i)
                    
                    image_conv1 = sess.run(nets['conv1_weight_image'])
                    train_writer.add_summary(image_conv1, i)
                    
                    print()
                    print("loop {0}, accuracy {1}".format(i,correct/total_count))
                    #print(start)
                    #print(end)
                    #print(total_count)
                    #print(correct)
                    #print("actual: ", indeces_list)
                    #print("predicted :", potato)
                    #print()
                    #f= open("output3.csv","a")
                    #f.write("loop {0}, accuracy {1}\n".format(i,correct/total_count))
                    #f.close()
    
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#builds the components
def conv2d(inputtensor, Weights, bias, strides=1,name = None):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(inputtensor, Weights, strides=[1, strides, strides, 1], padding='SAME',name=name)
    x = tf.nn.bias_add(x, bias)
    return tf.nn.relu(x)

def maxpool2x2(x, k=2, name = None):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME',name=name)
    
main()


In [ ]:
import tensorflow as tf
import numpy as np

out = tf.Variable( [[0,50,0,99], [0,50,1,0],[999,50,0,99]])

actual = tf.Variable([[0,1,0,1], [0,1,1,0],[1,0,0,0]])

condition = tf.equal(actual,1)


_,indeces = tf.nn.top_k(out,2)

preds = tf.Variable([[8,3], [1,2],[2,3]])
targ = tf.Variable([[8,3], [2,1],[2,3]])

results = tf.shape(tf.sets.set_intersection(indeces,targ,False).values)[0]


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("indeces ",sess.run(indeces))
    print("results", sess.run(results))
    
    

In [ ]:
a = np.array([[0,1,0,1], [0,1,1,0],[1,0,0,0]])
indeces = []
for row in a:
    indeces.append(np.where(row == 1)[0])

print(indeces)

In [ ]:
import h5py
import numpy as np
trainfile = "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_Coch.hdf5"
train_averagefile = "/om/user/ribeirop/audiosetDL/eval_stripped/eval_segments_average.npy"

trainset = h5py.File(trainfile)
start = 0
end = 5
trainBatchLabels = trainset["/labels"][start:end]

indeces_list = []
for row in trainBatchLabels:
    indeces= np.where(row==1)[0]
    print(indeces)
    indeces_list.append(np.pad(indeces,[0,15-len(indeces)],mode = 'constant',constant_values=-1))

print()
print(len(indeces_list[0]))

In [ ]:
#testing reshape for image
import tensorflow as tf
import numpy as np

from math import sqrt

from math import sqrt

def put_kernels_on_grid (kernel, pad = 1):

    '''Visualize conv. filters as an image (mostly for the 1st layer).
    Arranges filters into a grid, with some paddings between adjacent filters.

    Args:
    kernel:            tensor of shape [Y, X, NumChannels, NumKernels]
    pad:               number of black pixels around each filter (between them)

    Return:
    Tensor of shape [1, (Y+2*pad)*grid_Y, (X+2*pad)*grid_X, NumChannels].
    '''
    # get shape of the grid. NumKernels == grid_Y * grid_X
    def factorization(n):
        for i in range(int(sqrt(float(n))), 0, -1):
            if n % i == 0:
                if i == 1: print('Who would enter a prime number of filters')
                return (i, int(n / i))
    (grid_Y, grid_X) = factorization (kernel.get_shape()[3].value)
    print ('grid: %d = (%d, %d)' % (kernel.get_shape()[3].value, grid_Y, grid_X))

    x_min = tf.reduce_min(kernel)
    x_max = tf.reduce_max(kernel)
    kernel = (kernel - x_min) / (x_max - x_min)

    # pad X and Y
    x = tf.pad(kernel, tf.constant( [[pad,pad],[pad, pad],[0,0],[0,0]] ), mode = 'CONSTANT')

    # X and Y dimensions, w.r.t. padding
    Y = kernel.get_shape()[0] + 2 * pad
    X = kernel.get_shape()[1] + 2 * pad

    channels = kernel.get_shape()[2]

    # put NumKernels to the 1st dimension
    x = tf.transpose(x, (3, 0, 1, 2))
    # organize grid on Y axis
    x = tf.reshape(x, tf.stack([grid_X, Y * grid_Y, X, channels]))

    # switch X and Y axes
    x = tf.transpose(x, (0, 2, 1, 3))
    # organize grid on X axis
    x = tf.reshape(x, tf.stack([1, X * grid_X, Y * grid_Y, channels]))

    # back to normal order (not combining with the next step for clarity)
    x = tf.transpose(x, (2, 1, 3, 0))

    # to tf.image_summary order [batch_size, height, width, channels],
    #   where in this case batch_size == 1
    x = tf.transpose(x, (3, 0, 1, 2))

    # scaling to [0, 255] is not necessary for tensorboard
    return x


#
# ... and somewhere inside "def train():" after calling "inference()"
#

# Visualize conv1 kernels
#with tf.variable_scope('conv1'):
#  tf.get_variable_scope().reuse_variables()
#  weights = tf.get_variable('weights')
#  grid = put_kernels_on_grid (weights)
#  tf.image.summary('conv1/kernels', grid, max_outputs=1)



a = np.arange(81).reshape((3,3,1,9))

print(a)

first = tf.placeholder(tf.int32,[3,3,1,9], name = 'startShape')
second = put_kernels_on_grid(first, pad = 1)
#second = tf.reshape(first,[9,9])

with tf.Session() as sess:
    print(sess.run(second, feed_dict = {first:a}))
